In [1]:
import os
import gc
import PIL
import tqdm
import torch
import random
import shutil
import torchvision
import numpy as np
from torch import nn
from PIL import Image
import matplotlib.pyplot as plt
from torchsummary import summary
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [6]:
def print_shape(debug, to_print):
  if debug:
    print(to_print)

def double_conv_layers(in_channels, out_channels, kernel_size, activation=nn.ReLU, padding='same', batch_norm=True, coding_layer=False):
  '''
  Return Double Convolutional layers given the input parameters

  in_channels: input channels for the first convolutional layer
  out_channels: output channels for the second convolutional layer
  kernel_size: kernel size to use for both the layers
  activation: activaiton to apply to both the layers, should pass a activation function and not string.
  padding: padding to be applied to the inputs, by default no padding.
  batch_norm: if True applies nn.BatchNorm2d() after every Convolutional layer.
  '''

  if batch_norm:
    double_conv = nn.Sequential(
                                nn.Conv2d(in_channels, out_channels, kernel_size, padding=padding),
                                nn.BatchNorm2d(out_channels),
                                activation(inplace=True),
                                nn.Conv2d(out_channels, out_channels, kernel_size, padding=padding),
                                nn.BatchNorm2d(out_channels),
                                activation(inplace=True) if not coding_layer else CONFIG['coding_layer_activation']())
  else:
    double_conv = nn.Sequential(
                                nn.Conv2d(out_channels, out_channels, kernel_size, padding=padding),
                                activation(inplace=True),
                                nn.Conv2d(out_channels, out_channels, kernel_size, padding=padding),
                                activation(inplace=True) if not coding_layer else CONFIG['coding_layer_activation']())
  
  return double_conv

class ResidualUnit(nn.Module):
  def __init__(self, in_channels, out_channels, activation, kernel=3, first=False, debug=False):
    '''
    Description: Initializes the residual unit depending upon whether the residual unit is the first or the second unit in 
                 2-residual unit "layer"(A "layer" in ResNet is usually bunch of residual units with same kernel size).

    first : depending upon whether it is True or False the "first" or the "second" residual unit will be created
    overall_first : if the residual unit is overall the first residual unit of the network or not.
    '''
    super().__init__()

    self.first = first
    self.debug = debug

    if first:
      self.conv1 = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel, 2, padding=1),
                                 nn.BatchNorm2d(out_channels),
                                 activation())
      self.conv2 = nn.Sequential(nn.Conv2d(out_channels, out_channels, kernel, 1, padding='same'),
                                 nn.BatchNorm2d(out_channels))
      self.skipconv = nn.Sequential(nn.Conv2d(in_channels, out_channels, 1, 2, padding=0),
                                    nn.BatchNorm2d(out_channels))
      self.final_activation = activation()

    else:
      self.conv1 = nn.Sequential(nn.Conv2d(in_channels, in_channels, kernel, 1, padding='same'),
                                 nn.BatchNorm2d(in_channels),
                                 activation())
      self.conv2 = nn.Sequential(nn.Conv2d(in_channels, in_channels, kernel, 1, padding='same'),
                                 nn.BatchNorm2d(in_channels))
      self.final_activation = activation()

  def forward(self, input):
    
    print_shape(self.debug, '---------INSIDE RESIDUAL UNIT--------')

    x = self.conv1(input)
    print_shape(self.debug, f'After conv1 : {x.shape}')

    x = self.conv2(x)
    print_shape(self.debug, f'After conv2 : {x.shape}')

    if self.first:
      # if it is the first residual unit of the residual block, to match the image 
      # size before adding skip connection we have to process the image
      processed_input = self.skipconv(input)
      print_shape(self.debug, f'Processing skip input to shape: {processed_input.shape}')
      x = self.final_activation(x + processed_input) # skip connection
    
    else:
      x = self.final_activation(x + input) # skip connection
    
    print_shape(self.debug, f'Output shape for Residual Unit : {x.shape}')

    return x

class ResidualBlock(nn.Module):
  def __init__(self, in_channels, out_channels, activation, debug=False):
    super().__init__()

    self.debug = debug  

    self.ru1 = ResidualUnit(in_channels, out_channels, activation=activation, first=True, debug=self.debug)
    self.ru2 = ResidualUnit(out_channels, out_channels, activation=activation, debug=self.debug)

  def forward(self, input):
    x = self.ru1(input)
    x = self.ru2(x)
    return x


class ResNet18Encoder(nn.Module):
  def __init__(self, first_layer_kernel=7, activation=nn.ReLU, debug=False):
    super().__init__()

    self.debug = debug

    self.conv = nn.Conv2d(3, 64, first_layer_kernel, padding='same')

    self.residual_block_64 = ResidualBlock(64, 64, activation=activation, debug=self.debug) # 4 conv layers
    self.residual_block_128 = ResidualBlock(64, 128, activation=activation, debug=self.debug) # 4 conv layers
    self.residual_block_256 = ResidualBlock(128, 256, activation=activation, debug=self.debug)# 4 conv layers
    self.residual_block_512 = ResidualBlock(256, 512, activation=activation, debug=self.debug)# 4 conv layers
    
  def forward(self, input):
    
    print_shape(self.debug, f'Input shape : {input.shape}')
    print_shape(self.debug, f'-------ENCODER--------')

    x = self.conv(input)
    print_shape(self.debug, f'After first conv : {x.shape}')

    x1 = self.residual_block_64(x)
    x2 = self.residual_block_128(x1)
    x3 = self.residual_block_256(x2)
    x = self.residual_block_512(x3)

    return x, x3, x2, x1

class UnetDecoder(nn.Module):
  def __init__(self, activation=nn.ReLU, debug=False):
    super().__init__()
    
    self.debug = debug
     
    # combining upsample and conv2d layers as upsample_conv instead of using conv2dtranspose
    # as upsample is more stable for GAN architecture and does not create checkerboard artificats.
    self.upsample_conv_1 = nn.Sequential(nn.Upsample(scale_factor=2),
                                         nn.Conv2d(512, 256, 3, 1, padding='same'),
                                         activation())
    
    self.upsample_conv_2 = nn.Sequential(nn.Upsample(scale_factor=2),
                                         nn.Conv2d(256, 128, 3, 1, padding='same'),
                                         activation())
    
    self.upsample_conv_3 = nn.Sequential(nn.Upsample(scale_factor=2),
                                         nn.Conv2d(128, 64, 3, 1, padding='same'),
                                         activation())
    
    self.upsample_conv_4 = nn.Sequential(nn.Upsample(scale_factor=2),
                                         nn.Conv2d(64, 64, 3, 1, padding='same'),
                                         activation())
    
    self.up_conv_1 = double_conv_layers(512, 256, 3)
    self.up_conv_2 = double_conv_layers(256, 128, 3)
    self.up_conv_3 = double_conv_layers(128, 64, 3)

    self.final_conv = nn.Conv2d(64, 3, 1, padding='same')

  def forward(self, input, x3, x2, x1):
    
    print_shape(self.debug, f'-------DECODER-------')
   
    x = self.upsample_conv_1(input)
    print_shape(self.debug, f'After upsample 1 : {x.shape}')

    x = self.up_conv_1(torch.cat([x, x3], 1)) # skip connection
    print_shape(self.debug, f'After up conv 1 : {x.shape}')

    x = self.upsample_conv_2(x)
    print_shape(self.debug, f'After upsample 2 : {x.shape}')

    x = self.up_conv_2(torch.cat([x, x2], 1)) # skip connection
    print_shape(self.debug, f'After up conv 2 : {x.shape}')

    x = self.upsample_conv_3(x)
    print_shape(self.debug, f'After upsample 3 : {x.shape}')

    x = self.up_conv_3(torch.cat([x, x1], 1)) # skip connection
    print_shape(self.debug, f'After up conv 3 : {x.shape}')

    x = self.upsample_conv_4(x)
    print_shape(self.debug, f'After upsample 4 : {x.shape}')

    x = self.final_conv(x)
    print_shape(self.debug, f'After final conv : {x.shape}')

    return x

class ResNetUNet(nn.Module):
  def __init__(self, debug=False):
    super().__init__()
    
    self.debug =  debug
    
    self.encoder = ResNet18Encoder(debug=self.debug)
    self.decoder = UnetDecoder(debug=self.debug)

  def forward(self, input):

    x, x3, x2, x1 = self.encoder(input)
    x = self.decoder(x, x3, x2, x1)

    return x


In [4]:
image = torch.zeros(1, 3, 128, 128)
resnet = ResNet18Encoder(debug=True)
x = resnet(image)

Input shape : torch.Size([1, 3, 128, 128])
After first conv : torch.Size([1, 64, 128, 128])
---------INSIDE RESIDUAL UNIT--------
After conv1 : torch.Size([1, 64, 64, 64])
After conv2 : torch.Size([1, 64, 64, 64])
Processing skip input to shape: torch.Size([1, 64, 64, 64])
Output shape for Residual Unit : torch.Size([1, 64, 64, 64])
---------INSIDE RESIDUAL UNIT--------
After conv1 : torch.Size([1, 64, 64, 64])
After conv2 : torch.Size([1, 64, 64, 64])
Output shape for Residual Unit : torch.Size([1, 64, 64, 64])
---------INSIDE RESIDUAL UNIT--------
After conv1 : torch.Size([1, 128, 32, 32])
After conv2 : torch.Size([1, 128, 32, 32])
Processing skip input to shape: torch.Size([1, 128, 32, 32])
Output shape for Residual Unit : torch.Size([1, 128, 32, 32])
---------INSIDE RESIDUAL UNIT--------
After conv1 : torch.Size([1, 128, 32, 32])
After conv2 : torch.Size([1, 128, 32, 32])
Output shape for Residual Unit : torch.Size([1, 128, 32, 32])
---------INSIDE RESIDUAL UNIT--------
After conv1

In [14]:
image = torch.zeros(1, 512, 8, 8)
x3 = torch.zeros(1, 256, 16, 16)
x2 = torch.zeros(1, 128, 32, 32)
x1 = torch.zeros(1, 64, 64, 64)
decoder = UnetDecoder(debug=True)
decoder(image, x3, x2, x1)

-------DECODER-------
After upsample 1 : torch.Size([1, 256, 16, 16])
After up conv 1 : torch.Size([1, 256, 16, 16])
After upsample 2 : torch.Size([1, 128, 32, 32])
After up conv 2 : torch.Size([1, 128, 32, 32])
After upsample 3 : torch.Size([1, 64, 64, 64])
After up conv 3 : torch.Size([1, 64, 64, 64])
After upsample 4 : torch.Size([1, 64, 128, 128])
After final conv : torch.Size([1, 3, 128, 128])


In [7]:
image = torch.zeros(1, 3, 128, 128)
autoencoder = ResNetUNet(debug=True)
x = autoencoder(image)

Input shape : torch.Size([1, 3, 128, 128])
-------ENCODER--------
After first conv : torch.Size([1, 64, 128, 128])
---------INSIDE RESIDUAL UNIT--------
After conv1 : torch.Size([1, 64, 64, 64])
After conv2 : torch.Size([1, 64, 64, 64])
Processing skip input to shape: torch.Size([1, 64, 64, 64])
Output shape for Residual Unit : torch.Size([1, 64, 64, 64])
---------INSIDE RESIDUAL UNIT--------
After conv1 : torch.Size([1, 64, 64, 64])
After conv2 : torch.Size([1, 64, 64, 64])
Output shape for Residual Unit : torch.Size([1, 64, 64, 64])
---------INSIDE RESIDUAL UNIT--------
After conv1 : torch.Size([1, 128, 32, 32])
After conv2 : torch.Size([1, 128, 32, 32])
Processing skip input to shape: torch.Size([1, 128, 32, 32])
Output shape for Residual Unit : torch.Size([1, 128, 32, 32])
---------INSIDE RESIDUAL UNIT--------
After conv1 : torch.Size([1, 128, 32, 32])
After conv2 : torch.Size([1, 128, 32, 32])
Output shape for Residual Unit : torch.Size([1, 128, 32, 32])
---------INSIDE RESIDUAL U